In [2]:
# you will be prompted with a window asking to grant permissions
from google.colab import drive
drive.mount("/content/drive")

Mounted at /content/drive


In [3]:
# fill in the path in your Google Drive in the string below. Note: do not escape slashes or spaces
import os
datadir = "/content/第4章-数据评估与清洗"
if not os.path.exists(datadir):
  !ln -s "/content/drive/MyDrive/Python_train/第4章-数据评估与清洗" $datadir # TODO: Fill your Assignment 4 path
os.chdir(datadir)
!pwd

/content/drive/MyDrive/Python_train/第4章-数据评估与清洗


# 项目：评估和清理英国电商公司销售数据

## 分析目标

此数据分析的目的是，根据市场销售数据，挖掘畅销产品，以便制定更有效的市场策略来提升营收。

本实战项目的目的在于练习评估数据干净和整洁度，并且基于评估结果，对数据进行清洗，从而得到可供下一步分析的数据。

## 简介

原始数据集记录了一家英国在线零售公司在2010年12月1日至2011年12月9日期间的所有交易情况，涵盖了该公司在全球不同国家和地区的业务数据。该公司主要销售覆盖各个场景的礼品，包括但不限于生日礼品、结婚纪念品、圣诞礼品等等。该公司的客户群体主要包括批发商和个人消费者，其中批发商占据了相当大的比例。

数据每列的含义如下：
- `InvoiceNo`: 发票号码。6位数，作为交易的唯一标识符。如果这个代码以字母“c”开头，表示这笔交易被取消。
- `StockCode`: 产品代码。5位数，作为产品的唯一标识符。
- `Description`: 产品名称。
- `Quantity`: 产品在交易中的数量。
- `InvoiceDate`: 发票日期和时间。交易发生的日期和时间。
- `UnitPrice`: 单价。价格单位为英镑（£）。
- `CustomerID`: 客户编号。5位数，作为客户的唯一标识符。
- `Country`: 国家名称。客户所居住的国家的名称。

## 1.读取数据

导入数据分析所需要的库  
并通过Pandas的`read_csv`函数，将原始数据文件"e_commerce.csv"里的数据内容，解析为DataFrame，并赋值给变量`original_data`

In [4]:
import pandas as pd

original_data = pd.read_csv("e_commerce.csv")
original_data.head(5)

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,12/1/2010 8:26,2.55,17850.0,United Kingdom
1,536365,71053,WHITE METAL LANTERN,6,12/1/2010 8:26,3.39,17850.0,United Kingdom
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,12/1/2010 8:26,2.75,17850.0,United Kingdom
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,12/1/2010 8:26,3.39,17850.0,United Kingdom
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,12/1/2010 8:26,3.39,17850.0,United Kingdom


## 2.评估数据

- 评估：结构（整齐度）&内容（干净度）
- 数据的结构性问题指不符合“每列是一个变量，每行是一个观察值，每个单元格是一个值”这三个标准
- 数据的内容性问题包括存在丢失数据、重复数据、无效数据等。

### 2.1 评估结构（整齐度）

In [ ]:
original_data.sample(10) # 抽样看看

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
11686,537262,22867,HAND WARMER BIRD DESIGN,2,12/6/2010 11:26,2.10,15039.0,United Kingdom
288724,562204,20982,12 PENCILS TALL TUBE SKULLS,1,8/3/2011 12:54,0.85,16324.0,United Kingdom
184157,552677,21930,JUMBO STORAGE BAG SKULLS,3,5/10/2011 15:07,4.96,NaN,United Kingdom
474402,576840,23213,STAR WREATH DECORATION WITH BELL,5,11/16/2011 15:23,2.46,NaN,United Kingdom
196259,553833,23082,SET 6 PAPER TABLE LANTERN HEARTS,2,5/19/2011 11:41,3.75,17346.0,United Kingdom
132040,547652,21311,SET/4 BIRD MIRROR MAGNETS,1,3/24/2011 12:15,2.10,14606.0,United Kingdom
283946,561820,21154,RED RETROSPOT OVEN GLOVE,3,7/29/2011 16:00,2.46,NaN,United Kingdom
530157,580819,22380,TOY TIDY SPACEBOY,20,12/6/2011 11:30,0.79,15318.0,United Kingdom
479713,577174,23321,SMALL WHITE HEART OF WICKER,2,11/18/2011 10:58,1.65,17841.0,United Kingdom
152448,549594,22780,LIGHT GARLAND BUTTERFILES PINK,48,4/11/2011 10:25,3.37,16029.0,United Kingdom


从抽样的10行数据数据来看，数据符合“每列是一个变量，每行是一个观察值，每个单元格是一个值”。
具体来看不存在结构性问题。
- 每行：关于某商品的一次交易
- 每列：交易相关的各个变量

### 2.2 评估数据（干净度）

#### 2.2.0 简单总览

In [ ]:
original_data.info() #看看每列的变量类型，非空缺值

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 541909 entries, 0 to 541908
Data columns (total 8 columns):
 #   Column       Non-Null Count   Dtype  
---  ------       --------------   -----  
 0   InvoiceNo    541909 non-null  object 
 1   StockCode    541909 non-null  object 
 2   Description  540455 non-null  object 
 3   Quantity     541909 non-null  int64  
 4   InvoiceDate  541909 non-null  object 
 5   UnitPrice    541909 non-null  float64
 6   CustomerID   406829 non-null  float64
 7   Country      541909 non-null  object 
dtypes: float64(2), int64(1), object(5)
memory usage: 33.1+ MB


- 从输出结果来看，数据共有541909条观察值
- `Description`、`CustomerID`变量存在缺失值
- `InvoiceDate`的数据类型应为日期
- `CustomerID`的数据类型应为字符串

#### 2.2.1 缺失数据

##### `Description`

In [ ]:
#筛选出Description列缺失的行
original_data[original_data["Description"].isnull()]

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
622,536414,22139,NaN,56,12/1/2010 11:52,0.0,NaN,United Kingdom
1970,536545,21134,NaN,1,12/1/2010 14:32,0.0,NaN,United Kingdom
1971,536546,22145,NaN,1,12/1/2010 14:33,0.0,NaN,United Kingdom
1972,536547,37509,NaN,1,12/1/2010 14:33,0.0,NaN,United Kingdom
1987,536549,85226A,NaN,1,12/1/2010 14:34,0.0,NaN,United Kingdom
...,...,...,...,...,...,...,...,...
535322,581199,84581,NaN,-2,12/7/2011 18:26,0.0,NaN,United Kingdom
535326,581203,23406,NaN,15,12/7/2011 18:31,0.0,NaN,United Kingdom
535332,581209,21620,NaN,6,12/7/2011 18:35,0.0,NaN,United Kingdom
536981,581234,72817,NaN,27,12/8/2011 10:33,0.0,NaN,United Kingdom


- 发现有1454条交易数据缺失`Description`变量值
- 观察还发现，这些缺失`Description`的交易数据，`UnitPrice`都为0
- 为了验证猜想，我们增加筛选条件，看是否存在`Description`变量缺失且`UnitPrice`不为0的数据

In [ ]:
original_data[original_data["Description"].isnull() & original_data["UnitPrice"]!=0]

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country


- 筛选后发现，没有`Description`变量缺失且`UnitPrice`不为0的数据
- 也就是说，`Description`变量缺失的行`UnitPrice`都为0

进一步分析，
- `Description`表示产品名称，`UnitPrice`表示产品单价，都是进行后续商品交易分析的重要变量。如果它们同时缺失/无效，我们认为数据无法提供有效含义，因此这些后续可以被删除。

##### `CustomerID`

回到缺失数据的评估，上面提到`CustomerID`变量也存在缺失值

In [ ]:
original_data[original_data["CustomerID"].isnull()]

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
622,536414,22139,NaN,56,12/1/2010 11:52,0.00,NaN,United Kingdom
1443,536544,21773,DECORATIVE ROSE BATHROOM BOTTLE,1,12/1/2010 14:32,2.51,NaN,United Kingdom
1444,536544,21774,DECORATIVE CATS BATHROOM BOTTLE,2,12/1/2010 14:32,2.51,NaN,United Kingdom
1445,536544,21786,POLKADOT RAIN HAT,4,12/1/2010 14:32,0.85,NaN,United Kingdom
1446,536544,21787,RAIN PONCHO RETROSPOT,2,12/1/2010 14:32,1.66,NaN,United Kingdom
...,...,...,...,...,...,...,...,...
541536,581498,85099B,JUMBO BAG RED RETROSPOT,5,12/9/2011 10:26,4.13,NaN,United Kingdom
541537,581498,85099C,JUMBO BAG BAROQUE BLACK WHITE,4,12/9/2011 10:26,4.13,NaN,United Kingdom
541538,581498,85150,LADIES & GENTLEMEN METAL SIGN,1,12/9/2011 10:26,4.96,NaN,United Kingdom
541539,581498,85174,S/4 CACTI CANDLES,1,12/9/2011 10:26,10.79,NaN,United Kingdom


- 发现有135080条交易数据缺失`CustomerID`变量值
- `CustomerID`表示客户编号，不是分析畅销商品的必要变量。并且从输出结果来看，有些`CustomerID`缺失的交易数据仍然有效，因此保留此变量为空的观察值。

#### 2.2.2 重复数据

根据数据变量的含义来看，虽然`InvoiceNo`、`StockCode`和`CustomerID`都是唯一标识符，但一次交易可能包含多件商品，因此`InvoiceNo`可以存在重复；不同交易可以包含同件商品，因此`StockCode`可以存在重复；顾客可以进行多次交易或下单多个商品，因此`CustomerID`也可以存在重复。

那么针对此数据集，我们无需评估重复数据。

#### 2.2.3 不一致数据

不一致数据可能存在于`Country`变量中，我们要查看是否存在多个不同值指代同一国家的情况。

In [ ]:
original_data["Country"].value_counts()

United Kingdom          495266
Germany                   9495
France                    8557
EIRE                      8196
Spain                     2533
Netherlands               2371
Belgium                   2069
Switzerland               2002
Portugal                  1519
Australia                 1259
Norway                    1086
Italy                      803
Channel Islands            758
Finland                    695
Cyprus                     622
Sweden                     462
Unspecified                446
Austria                    401
Denmark                    389
Japan                      358
Poland                     341
Israel                     297
China                      288
Singapore                  229
USA                        218
UK                         211
Iceland                    182
Canada                     151
Greece                     146
Malta                      127
United States               73
United Arab Emirates        68
European

从`Country`变量值来看，
- `"USA"`、`"United States"`均在表示美国
- `"United Kingdom"`、`"UK"`、`"U.K."`均在表示英国  
因此应该在清洗步骤对这些值进行统一，只保留一个指代值。

#### 2.2.4 无效/错误数据

In [ ]:
original_data.describe()

,Quantity,UnitPrice,CustomerID
count,541909.000000,541909.000000,406829.000000
mean,9.552250,4.611114,15287.690570
std,218.081158,96.759853,1713.600303
min,-80995.000000,-11062.060000,12346.000000
25%,1.000000,1.250000,13953.000000
50%,3.000000,2.080000,15152.000000
75%,10.000000,4.130000,16791.000000
max,80995.000000,38970.000000,18287.000000


- 从输出结果来看，`Quantity`和`UnitPrice`存在负数，会对后续数值分析产生影响。
- 因此，我们先筛选出`Quantity`数值为负数的观察值，进一步评估其含义。

##### Quantity

In [ ]:
original_data[original_data["Quantity"]<0]

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
141,C536379,D,Discount,-1,12/1/2010 9:41,27.50,14527.0,United Kingdom
154,C536383,35004C,SET OF 3 COLOURED FLYING DUCKS,-1,12/1/2010 9:49,4.65,15311.0,United Kingdom
235,C536391,22556,PLASTERS IN TIN CIRCUS PARADE,-12,12/1/2010 10:24,1.65,17548.0,United Kingdom
236,C536391,21984,PACK OF 12 PINK PAISLEY TISSUES,-24,12/1/2010 10:24,0.29,17548.0,United Kingdom
237,C536391,21983,PACK OF 12 BLUE PAISLEY TISSUES,-24,12/1/2010 10:24,0.29,17548.0,United Kingdom
...,...,...,...,...,...,...,...,...
540449,C581490,23144,ZINC T-LIGHT HOLDER STARS SMALL,-11,12/9/2011 9:57,0.83,14397.0,United Kingdom
541541,C581499,M,Manual,-1,12/9/2011 10:28,224.69,15498.0,United Kingdom
541715,C581568,21258,VICTORIAN SEWING BOX LARGE,-5,12/9/2011 11:57,10.95,15311.0,United Kingdom
541716,C581569,84978,HANGING HEART JAR T-LIGHT HOLDER,-1,12/9/2011 11:58,1.25,17315.0,United Kingdom


从筛选结果来看，`Quantity`变量为负数的观察值，`InvoiceNo`似乎均以C开头，表示订单被取消。

但此猜测不一定准确，为了验证猜想，我们增加筛选条件，看是否存在`Quantity`变量为负数且`InvoiceNo`不以C开头的观察值。

In [ ]:
original_data[(original_data["Quantity"]<0) & (original_data["InvoiceNo"].str[0]!="C")]

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
2406,536589,21777,NaN,-10,12/1/2010 16:50,0.0,NaN,United Kingdom
4347,536764,84952C,NaN,-38,12/2/2010 14:42,0.0,NaN,United Kingdom
7188,536996,22712,NaN,-20,12/3/2010 15:30,0.0,NaN,United Kingdom
7189,536997,22028,NaN,-20,12/3/2010 15:30,0.0,NaN,United Kingdom
7190,536998,85067,NaN,-6,12/3/2010 15:30,0.0,NaN,United Kingdom
...,...,...,...,...,...,...,...,...
535333,581210,23395,check,-26,12/7/2011 18:36,0.0,NaN,United Kingdom
535335,581212,22578,lost,-1050,12/7/2011 18:38,0.0,NaN,United Kingdom
535336,581213,22576,check,-30,12/7/2011 18:38,0.0,NaN,United Kingdom
536908,581226,23090,missing,-338,12/8/2011 9:56,0.0,NaN,United Kingdom


以上猜想被证实错误，因为还存在`Quantity`变量为负数且`InvoiceNo`不以C开头的观察值。

但根据以上输出结果，这些筛选出的观察值的`UnitPrice`观察值均为0，因此增加`UnitPrice`的条件进行验证。

In [ ]:
original_data[(original_data["Quantity"]<0) & (original_data["InvoiceNo"].str[0]!="C") & (original_data["UnitPrice"]!=0)]

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country


根据输出结果，猜想得到验证，当`Quantity`变量为负数的时候，观察值满足以下条件之一：
- `InvoiceNo`以C开头，表示订单被取消
- `UnitPrice`为0，说明单价为0英镑

这些交易数据均不是有效成交数据，不贡献销售，不在后续分析范围内，因此我们将在数据清理步骤，将`Quantity`为负数的观察值删除。

##### UnitPrice

In [5]:
original_data[original_data["UnitPrice"]<0]

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
299983,A563186,B,Adjust bad debt,1,8/12/2011 14:51,-11062.06,NaN,United Kingdom
299984,A563187,B,Adjust bad debt,1,8/12/2011 14:52,-11062.06,NaN,United Kingdom


从输出结果来看，UnitPrice为负数的观察值都是坏账调账，不属于实际商品交易数据，因此也在数据清理步骤中也将其删除。

## 3.清理数据

根据前面评估部分得到的结论，我们需要进行的数据清理包括：
- 把`InvoiceDate`变量的数据类型转换为为日期时间
- 把`CustomerID`变量的数据类型转换为字符串
- 把`Description`变量缺失的观察值删除
- 把`Country`变量值`"USA"`替换为`"United States"`
- 把`Country`变量值`"UK"`、`"U.K."`替换为`"United Kingdom"`
- 把`Quantity`变量值为负数的观察值删除
- 把`UnitPrice`变量值为负数的观察值删除

为了区分开经过清理的数据和原始的数据，我们创建新的变量`cleaned_data`，让它为`original_data`复制出的副本。我们之后的清理步骤都将被运用在`cleaned_data`上。

In [6]:
cleaned_data = original_data.copy()
cleaned_data.head(5)

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,12/1/2010 8:26,2.55,17850.0,United Kingdom
1,536365,71053,WHITE METAL LANTERN,6,12/1/2010 8:26,3.39,17850.0,United Kingdom
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,12/1/2010 8:26,2.75,17850.0,United Kingdom
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,12/1/2010 8:26,3.39,17850.0,United Kingdom
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,12/1/2010 8:26,3.39,17850.0,United Kingdom


- 把`InvoiceDate`变量的数据类型转换为为日期时间

In [12]:
cleaned_data["InvoiceDate"]=pd.to_datetime(cleaned_data["InvoiceDate"])
cleaned_data["InvoiceDate"]

,InvoiceDate
0,2010-12-01 08:26:00
1,2010-12-01 08:26:00
2,2010-12-01 08:26:00
3,2010-12-01 08:26:00
4,2010-12-01 08:26:00
...,...
541904,2011-12-09 12:50:00
541905,2011-12-09 12:50:00
541906,2011-12-09 12:50:00
541907,2011-12-09 12:50:00


- 把`CustomerID`变量的数据类型转换为字符串

In [14]:
cleaned_data["CustomerID"]=cleaned_data["CustomerID"].astype(str)
cleaned_data["CustomerID"]

,CustomerID
0,17850.0
1,17850.0
2,17850.0
3,17850.0
4,17850.0
...,...
541904,12680.0
541905,12680.0
541906,12680.0
541907,12680.0


- 把`CustomerID`变量值结尾的`".0"`删除

In [17]:
cleaned_data["CustomerID"] = cleaned_data["CustomerID"].str.slice(0, -2)
cleaned_data["CustomerID"]

,CustomerID
0,17850
1,17850
2,17850
3,17850
4,17850
...,...
541904,12680
541905,12680
541906,12680
541907,12680


- 把`Description`变量缺失的观察值删除

In [16]:
cleaned_data.dropna(subset=["Description"],inplace=True)
cleaned_data[cleaned_data["Description"].isnull()]

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country


In [18]:
cleaned_data["Description"].isnull().sum()

np.int64(0)

- 把`Country`变量值`"USA"`替换为`"United States"`
- 把`Country`变量值`"UK"`、`"U.K."`替换为`"United Kingdom"`

In [20]:
cleaned_data["Country"].replace({"USA":"United States", "UK":"United Kingdom", "U.K.":"United Kingdom"}, inplace=True)
cleaned_data["Country"].value_counts()

,count
Country,
United Kingdom,494024
Germany,9495
France,8557
EIRE,8196
Spain,2533
Netherlands,2371
Belgium,2069
Switzerland,2002
Portugal,1519


In [21]:
print(len(cleaned_data[cleaned_data["Country"] == "USA"]))
print(len(cleaned_data[cleaned_data["Country"] == "UK"]))
print(len(cleaned_data[cleaned_data["Country"] == "U.K."]))

0
0
0


- 把`Quantity`变量值为负数的观察值删除

In [24]:
cleaned_data = cleaned_data[cleaned_data["Quantity"]>=0]
cleaned_data[cleaned_data["Quantity"]<0]

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country


- 把`UnitPrice`变量值为负数的观察值删除

In [25]:
cleaned_data = cleaned_data[cleaned_data["UnitPrice"]>=0]
cleaned_data[cleaned_data["UnitPrice"]<0]

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country


## 4.保存清理后的数据

In [27]:
cleaned_data.to_csv("e_commerce_cleaned.csv", index=False)

In [30]:
pd.read_csv("e_commerce_cleaned.csv").head(5)

<ipython-input-30-e4f9bb6e9125>:1: DtypeWarning: Columns (0) have mixed types. Specify dtype option on import or set low_memory=False.
  pd.read_csv("e_commerce_cleaned.csv").head(5)


,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,2010-12-01 08:26:00,2.55,17850,United Kingdom
1,536365,71053,WHITE METAL LANTERN,6,2010-12-01 08:26:00,3.39,17850,United Kingdom
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,2010-12-01 08:26:00,2.75,17850,United Kingdom
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,2010-12-01 08:26:00,3.39,17850,United Kingdom
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,2010-12-01 08:26:00,3.39,17850,United Kingdom
